# 本題要找出各部門最高薪水的人為何 (\#Pandas)
原題目連結: https://leetcode.com/problems/department-highest-salary/description/?envType=study-plan-v2&envId=30-days-of-pandas&lang=pythondata  

Table: Employee  
```
+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| id           | int     |
| name         | varchar |
| salary       | int     |
| departmentId | int     |
+--------------+---------+
```
id is the primary key (column with unique values) for this table.  
departmentId is a foreign key (reference columns) of the ID from the Department table.  
Each row of this table indicates the ID, name, and salary of an employee. It also contains the ID of their department.  

Table: Department  
```
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| name        | varchar |
+-------------+---------+
```
id is the primary key (column with unique values) for this table. It is guaranteed that department name is not NULL.  
Each row of this table indicates the ID of a department and its name.  
 
Write a solution to find employees who have the highest salary in each of the departments.  

Return the result table in any order.  

The result format is in the following example.  

範例:  
Ex1:  
Input:   
Employee table:    
```
+----+-------+--------+--------------+
| id | name  | salary | departmentId |
+----+-------+--------+--------------+
| 1  | Joe   | 70000  | 1            |
| 2  | Jim   | 90000  | 1            |
| 3  | Henry | 80000  | 2            |
| 4  | Sam   | 60000  | 2            |
| 5  | Max   | 90000  | 1            |
+----+-------+--------+--------------+
```
Department table:  
```
+----+-------+
| id | name  |
+----+-------+
| 1  | IT    |
| 2  | Sales |
+----+-------+
```
Output:  
```
+------------+----------+--------+
| Department | Employee | Salary |
+------------+----------+--------+
| IT         | Jim      | 90000  |
| Sales      | Henry    | 80000  |
| IT         | Max      | 90000  |
+------------+----------+--------+
```
Explanation: Max and Jim both have the highest salary in the IT department and Henry has the highest salary in the Sales department.

* 解題想法:  
首先找出每個部門最高的薪水與人分別為何，接著將這些資料與部門名稱串在一起，最後回傳一個新的DataFrame就是答案

In [ ]:
import pandas as pd

def department_highest_salary(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    employee = employee[['name','salary','departmentId']].rename(columns = {'departmentId':'id', 'name': 'Employee', 'salary':'Salary'})
    salary = pd.merge(employee, department, how = 'left', on='id').drop(columns=['id']).rename(columns = {'name':'Department'})
    for i in range(len(department)):
        d = department.iloc[i]['name']
        s = salary.query('Department == "{}"'.format(d)).nlargest(1, 'Salary').nlargest(1, 'Salary')
        if len(s) > 0: 
            h = s.iloc[0]['Salary']
            p = salary.query('Department == "{}" & Salary == {}'.format(d, h))
            salary = salary.query('Department != "{}"'.format(d))
            salary = pd.concat([salary, p])
    return salary